In [1]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torchvision.utils import save_image


import import_ipynb

from CustomDataset import ControlsDataset


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
print('using device', device)
torch.cuda.empty_cache()

importing Jupyter notebook from CustomDataset.ipynb
using device cuda


In [2]:
dataset = ControlsDataset()
dataset.make_dataloaders()

Total training stacks 7
Total validation stacks 2


In [3]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder,self).__init__()
        img_size = torch.Size([1, 3, 170, 640]) # [batch_size, channels, height, width]
        empty = torch.zeros(img_size)
        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(True),
            nn.Conv2d(6,16,kernel_size=5),
            nn.ReLU(True))
        
        
        units = self.encoder(empty).numel()
        print(units)
        
        self.latent = nn.Sequential(nn.Linear(units, 2800))
        
        self.decoder = nn.Sequential(             
            nn.ConvTranspose2d(16,6,kernel_size=5),
            nn.ReLU(True),
            nn.ConvTranspose2d(6,3,kernel_size=5),
            nn.ReLU(True))
        
        
    def forward(self,x):
        batch_size = x.shape[0]
        x = self.encoder(x)
        x = x.reshape((batch_size,-1))
        #print(x.shape)
        x = self.latent(x)
        print(x.shape)
        x = x.reshape(batch_size,16,7,25)
        x = self.decoder(x)
        #print(x.shape)
        return x

In [4]:
num_epochs = 5 #you can go for more epochs, I am using a mac
batch_size = 128

In [5]:
model = Autoencoder()
distance = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-5)

1638144


In [6]:
for epoch in range(num_epochs):
    for data in dataset.dataloader:
        img = data['image'].float()
        img = torch.Tensor(img)
        # ===================forward=====================
        output = model(img)
        print(output.shape)
        loss = distance(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss))

torch.Size([128, 2800])
torch.Size([128, 3, 15, 33])


RuntimeError: The size of tensor a (33) must match the size of tensor b (640) at non-singleton dimension 3

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
for epoch in range(num_epochs):
    for data in dataset.dataloader:
        img = data['image'].float()
        first = img[0]
        output = model(img).detach().numpy()[0]
        first = np.transpose(first,(1,2,0))
        output = np.transpose(output,(1,2,0))
        plt.imshow(first)
        plt.show()
        plt.imshow(output)
        plt.show()
        break
    break
        